<a href="https://colab.research.google.com/github/AlvaroUriel/idal_ia3/blob/main/tfm_v1_1_061222.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from  google.colab import drive
 
drive.mount('/content/drive/')

Mounted at /content/drive/


In [52]:
import numpy as np
import pandas as pd
import xml.etree.ElementTree as ET

import matplotlib.pyplot as plt; 
from tensorflow.keras.utils import load_img, img_to_array
import matplotlib.patches as patches
import os
import shutil

In [53]:
os.listdir('/content/drive/MyDrive/IA3/TFM/archive/YOLO/')

['annotations1',
 'images1',
 'images0',
 'images',
 'labels0',
 'labels',
 'annotations0',
 'annotations']

In [ ]:
#!cp -r /content/drive/MyDrive/IA3/TFM/archive/images /content/drive/MyDrive/IA3/TFM/archive/YOLO

In [ ]:
#!cp -r /content/drive/MyDrive/IA3/TFM/archive/annotations /content/drive/MyDrive/IA3/TFM/archive/YOLO

In [54]:
# Convert Pascal_Voc bb to Yolo
def pascal_voc_to_yolo(x1, y1, x2, y2, image_w, image_h):
    return [((x2 + x1)/(2*image_w)), ((y2 + y1)/(2*image_h)), (x2 - x1)/image_w, (y2 - y1)/image_h]

In [55]:
newlist=[]

In [56]:
for name in os.listdir('/content/drive/MyDrive/IA3/TFM/archive/croppedImages1'):
  newlist.append(name[:-4].split('_')[3])

In [57]:
#Solo copia una vez cada imagen de la carpeta de imagenes "limpiadas",
# da igual que este repetido el nombre solo deja una copia
for name in newlist:
  pathImg='/content/drive/MyDrive/IA3/TFM/archive/images/' + name + '.png'
  shutil.copyfile(pathImg, '/content/drive/MyDrive/IA3/TFM/archive/YOLO/images1/'+ name + '.png')
  
  pathAnnot='/content/drive/MyDrive/IA3/TFM/archive/annotations/' + name + '.xml'
  shutil.copyfile(pathAnnot, '/content/drive/MyDrive/IA3/TFM/archive/YOLO/annotations1/'+ name + '.xml')

In [ ]:
#!mkdir /content/drive/MyDrive/IA3/TFM/archive/YOLO/labels1/

mkdir: cannot create directory ‘/content/drive/MyDrive/IA3/TFM/archive/YOLO/labels/’: File exists


In [58]:
#Generamos todos los labels apartir de las annotations
bGenLabels = 0
if(bGenLabels):

  xml_path      = '/content/drive/MyDrive/IA3/TFM/archive/YOLO/annotations1/'
  txt_path      = '/content/drive/MyDrive/IA3/TFM/archive/YOLO/labels1/'

  contenido = os.listdir(xml_path)

  for fichero in contenido:
    if fichero.endswith('.xml'):
      
      tree=ET.parse(xml_path+fichero)

      imgW = int (tree.find('size').find('width').text)
      imgH = int (tree.find('size').find('height').text)

      nuevo_archivo = open(txt_path+fichero[:-3]+'txt', "w")

      for element in tree.findall('object'):

        xmin = int(element.find("bndbox").find('xmin').text)
        ymin = int(element.find("bndbox").find('ymin').text)
        xmax = int(element.find("bndbox").find('xmax').text)
        ymax = int(element.find("bndbox").find('ymax').text)

        p = pascal_voc_to_yolo(xmin, ymin, xmax, ymax, imgW, imgH)

        nuevo_archivo.write(str(0) + " " + str(p[0]) + " " + str(p[1]) + " " + str(p[2]) + " " + str(p[3]) + "\n")

      nuevo_archivo.close()  

In [61]:
#!mkdir /content/drive/MyDrive/IA3/TFM/archive/YOLO/train_data/
#!mkdir /content/drive/MyDrive/IA3/TFM/archive/YOLO/train_data/images/
#!mkdir /content/drive/MyDrive/IA3/TFM/archive/YOLO/train_data/images/train/
#!mkdir /content/drive/MyDrive/IA3/TFM/archive/YOLO/train_data/images/val/
#!mkdir /content/drive/MyDrive/IA3/TFM/archive/YOLO/train_data/labels/
#!mkdir /content/drive/MyDrive/IA3/TFM/archive/YOLO/train_data/labels/train/
#!mkdir /content/drive/MyDrive/IA3/TFM/archive/YOLO/train_data/labels/val/

mkdir: cannot create directory ‘/content/drive/MyDrive/IA3/TFM/archive/YOLO/train_data/’: File exists


In [62]:
#Generamos la estructura de directorios de entrenamiento
bSplit = 0
if(bSplit):

  img_path      = '/content/drive/MyDrive/IA3/TFM/archive/YOLO/images1/'
  txt_path      = '/content/drive/MyDrive/IA3/TFM/archive/YOLO/labels1/'

  img_train_path  = '/content/drive/MyDrive/IA3/TFM/archive/YOLO/train_data/images/train/'
  img_val_path    = '/content/drive/MyDrive/IA3/TFM/archive/YOLO/train_data/images/val/'

  txt_train_path  = '/content/drive/MyDrive/IA3/TFM/archive/YOLO/train_data/labels/train/'
  txt_val_path    = '/content/drive/MyDrive/IA3/TFM/archive/YOLO/train_data/labels/val/'

  contenido = os.listdir(img_path)
  percTrain = int(len(contenido)*70/100)

  cont=0

  for fichero in contenido:
    if fichero.endswith('.png'):      
      if(cont<percTrain):
        shutil.copyfile(img_path+fichero[:-3]+'png', img_train_path+fichero[:-3]+'png')
        shutil.copyfile(txt_path+fichero[:-3]+'txt', txt_train_path+fichero[:-3]+'txt')
      else:
        shutil.copyfile(img_path+fichero[:-3]+'png', img_val_path+fichero[:-3]+'png')
        shutil.copyfile(txt_path+fichero[:-3]+'txt', txt_val_path+fichero[:-3]+'txt')

      cont=cont+1

In [66]:
print(len(os.listdir(img_train_path)))
print(len(os.listdir(img_val_path)))
print(len(os.listdir(txt_train_path)))
print(len(os.listdir(txt_val_path)))

482
207
482
207
